In [1]:
from tkinter import *
from tkinter import ttk
import tkinter.font as tkFont
from tkinter import font

In [2]:
from movie import Movie
from parse import parse_imdb_data

In [3]:
import os
import pickle

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import svm

In [5]:
import numpy as np
from scipy.sparse import csr_matrix, vstack, hstack

In [6]:
if os.path.isfile("movies.p"):
    movies = pickle.load(open("movies.p", "rb"))
else:
    movies = parse_imdb_data()
    pickle.dump(movies, open( "movies.p", "wb" ))

In [7]:
m_ = movies
#m_=list(filter(lambda m: m.year>2000, m_))
m_=list(filter(lambda m: m.num_ratings>1000, m_))
m_=list(filter(lambda m: m.mean_rating>8.5, m_))
len(m_)

691

In [8]:
def sortby(tree, col, descending):
    """sort tree contents when a column header is clicked on"""
    # grab values to sort
    data = [(tree.set(child, col), child) \
        for child in tree.get_children('')]
    # if the data to be sorted is numeric change to float
    #data =  change_numeric(data)
    # now sort the data in place
    if(data[0][0].isnumeric()): # FIX: doesn't work for float
        data = [(float(d[0]), d[1]) for d in data]
    data.sort(reverse=descending)
    for ix, item in enumerate(data):
        tree.move(item[1], '', ix)
    # switch the heading so it will sort in the opposite direction
    tree.heading(col, command=lambda col=col: sortby(tree, col, \
        int(not descending)))

In [9]:
min_year = min_rating_count = min_mean_rating = float('inf')
max_year = max_rating_count = max_mean_rating = float('-inf')
for m in m_:
    min_year = min((min_year, m.year))
    max_year = max((max_year, m.year))
    min_rating_count = min((min_rating_count, m.num_ratings))
    max_rating_count = max((max_rating_count, m.num_ratings))
    min_mean_rating = min((min_mean_rating, m.mean_rating))
    max_mean_rating = max((max_mean_rating, m.mean_rating))    

In [10]:
if os.path.isfile("ratings.p"):
    ratings = pickle.load(open("ratings.p", "rb"))
else:
    ratings = {}

In [11]:
ratings = {}

In [31]:
root = Tk()
root.title("IMDB MOVIE RECOMMENDER SYSTEM")

root.rowconfigure(0, weight=1)
root.columnconfigure(0, weight=1)

mainframe = ttk.Frame(root, padding="3 3 12 12")
mainframe.grid(column=0, row=0, sticky="nsew")
mainframe['borderwidth'] = 10
mainframe['relief'] = 'sunken'

mainframe.rowconfigure(0, weight=1)
mainframe.columnconfigure(0, weight=0)
mainframe.columnconfigure(1, weight=1)

filterframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Filter")
filterframe.grid(column=0, row=0, sticky="nsew")
filterframe['borderwidth'] = 2
filterframe['relief'] = 'sunken'


dataframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Data")
dataframe.grid(column=1, row=0, sticky="nsew")
dataframe['borderwidth'] = 2
dataframe['relief'] = 'sunken'

dataframe.grid_columnconfigure(0, weight=1)
dataframe.grid_columnconfigure(1, weight=0)
dataframe.grid_rowconfigure(0, weight=1)
dataframe.grid_rowconfigure(1, weight=0)
dataframe.grid_rowconfigure(2, weight=0)


yearfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Year")
yearfilterframe.grid(column=0, row=0, sticky="nsew")
yearfilterframe['borderwidth'] = 2
yearfilterframe['relief'] = 'sunken'
ttk.Label(yearfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minyear=IntVar()
minyear.set(min_year)
ttk.Entry(yearfilterframe, textvariable=minyear).grid(column=1, row=0, sticky="E")
ttk.Label(yearfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxyear=IntVar()
maxyear.set(max_year)
ttk.Entry(yearfilterframe, textvariable=maxyear).grid(column=1, row=1, sticky="E")


numratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="# of Ratings")
numratingfilterframe.grid(column=0, row=1, sticky="nsew")
numratingfilterframe['borderwidth'] = 2
numratingfilterframe['relief'] = 'sunken'
ttk.Label(numratingfilterframe, text="MIN").grid(column=0, row=0, sticky="E")
minratingcount=IntVar()
minratingcount.set(min_rating_count)
ttk.Entry(numratingfilterframe, textvariable=minratingcount).grid(column=1, row=0, sticky="W")
ttk.Label(numratingfilterframe, text="MAX").grid(column=0, row=1, sticky="E")
maxratingcount=IntVar()
maxratingcount.set(max_rating_count)
ttk.Entry(numratingfilterframe, textvariable=maxratingcount).grid(column=1, row=1, sticky="W")


meanratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Average Rating")
meanratingfilterframe.grid(column=0, row=2, sticky="nsew")
meanratingfilterframe['borderwidth'] = 2
meanratingfilterframe['relief'] = 'sunken'
ttk.Label(meanratingfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minrating=DoubleVar()
minrating.set(min_mean_rating)
ttk.Entry(meanratingfilterframe, textvariable=minrating).grid(column=1, row=0, sticky="E")
ttk.Label(meanratingfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxrating=DoubleVar()
maxrating.set(max_mean_rating)
ttk.Entry(meanratingfilterframe, textvariable=maxrating).grid(column=1, row=1, sticky="E")

phrasefilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Phrase")
phrasefilterframe.grid(column=0, row=3, sticky="nsew")
phrasefilterframe['borderwidth'] = 2
phrasefilterframe['relief'] = 'sunken'

titlephrase = StringVar()
ttk.Label(phrasefilterframe, text="Title").grid(column=0, row=0, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=titlephrase).grid(column=1, row=0, sticky="nsew")
keywordphrase = StringVar()
ttk.Label(phrasefilterframe, text="Keyword").grid(column=0, row=1, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=keywordphrase).grid(column=1, row=1, sticky="nsew")
genrephrase = StringVar()
ttk.Label(phrasefilterframe, text="Genre").grid(column=0, row=2, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=genrephrase).grid(column=1, row=2, sticky="nsew")

certfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="USA Certificate")
certfilterframe.grid(column=0, row=4, sticky="nsew")
certfilterframe['borderwidth'] = 2
certfilterframe['relief'] = 'sunken'


left_certificates = ["Approved", "Passed", "G", "PG", "PG-13", "R", "Unrated", "Not Rated"]
right_certificates = ["TV-G", "TV-PG", "TV-Y", "TV-Y7", "TV-14", "TV-MA"]
cert_vars = []
i=0

for cert in left_certificates:
    cv = StringVar()
    cv.set(cert)
    ttk.Checkbutton(certfilterframe, text=cert, onvalue = cert, variable=cv).grid(column=0, row=i, sticky="w")
    cert_vars.append(cv)
    i += 1

i=0

for cert in right_certificates:    
    cv = StringVar()
    cv.set(cert)
    ttk.Checkbutton(certfilterframe, text=cert, onvalue = cert, variable=cv).grid(column=2, row=i, sticky="w")
    cert_vars.append(cv)
    i += 1

cert_Unk = StringVar()
cert_Unk.set("Unknown")
ttk.Checkbutton(certfilterframe, text='Unknown', onvalue = "Unknown", variable=cert_Unk).grid(column=1, 
                                                                                            row=max((len(right_certificates),len(left_certificates)))+1, sticky="w")
cert_vars.append(cert_Unk)


removed_iids = set()

def filter_data(*args):
    remove_iids = set()
    data = data_tree.get_children('')
    #print(data_tree.set(data[0]))
    display_certificates = set()
    
    for cv in cert_vars:
        if cv.get() != "0":
            display_certificates.add(cv.get())
    
    
    for iid in data:
        m = m_[int(iid)]
        # Year
        if m.year < minyear.get():
            remove_iids.add(iid)
        if m.year > maxyear.get():
            remove_iids.add(iid)
        # Rating Count
        if m.num_ratings < minratingcount.get():
            remove_iids.add(iid)
        if m.num_ratings > maxratingcount.get():
            remove_iids.add(iid)
        # MEAN RATING
        if m.mean_rating < minrating.get():
            remove_iids.add(iid)
        if m.mean_rating > maxrating.get():
            remove_iids.add(iid)
        if m.title.find(titlephrase.get()) == -1:
            remove_iids.add(iid)
        
        if genrephrase.get() != "":
            if m.genres == None:
                remove_iids.add(iid)
            else:
                gp = False
                for g in m.genres:
                    if g.find(genrephrase.get()) != -1:
                        gp = True
                        break
                if not gp:
                    remove_iids.add(iid)
        if keywordphrase.get() != "":
            if m.keywords == None:
                remove_iids.add(iid)
            else:
                kp = False
                for k in m.keywords:
                    if k.find(keywordphrase.get()) != -1:
                        kp = True
                        break
                if not kp:
                    remove_iids.add(iid)
        
        if m.certificate not in display_certificates:
            remove_iids.add(iid)
        
    #data_tree.delete(list(remove_iids))
    for iid in remove_iids:
        data_tree.delete(iid)
    removed_iids.update(remove_iids)
    #TODO
    # 1. Update the other fields based on the new filtered data
    # 2. There is no way to go back in filtering right now

def reset_filters(*args):
    for iid in removed_iids:
        i = int(iid)
        item = [m_[i].title, m_[i].year, m_[i].num_ratings, m_[i].mean_rating, m_[i].certificate]
        data_tree.insert('', 'end', iid = str(i), values=item)
    sortby(data_tree, 'Title', False)
    removed_iids.clear()
    #TODO Reset the fields to their default values
    
def double_click(e):
    item = data_tree.selection()[0]
    #print("You clicked on", data_tree.set(item))
    #print(m_[int(item)])
    tl = Toplevel(root)
    #ttk.Label(tl, text=data_tree.set(item)).grid(column=0, row=0, sticky="E")
    m = m_[int(item)]
    
    def rate_movie(*args):
        #print(m.title)
        #print(rating.get())
        #ratings[m.title+str(m.year)]=rating.get()
        if rating.get() != "no-opinion":
            ratings[m] = rating.get()
        else:
            ratings.pop(m, None)
        tl.destroy()
    
    ttk.Label(tl, text=m.title + " (" + str(m.year) + ")", font=font.Font(family='Helvetica', size=14, weight='bold')).grid(column=0, row=0, sticky="W")
    if m.genres is not None:
        g = ", ".join(m.genres)
        ttk.Label(tl, text="Genres: " + g).grid(column=0, row=1, sticky="W")      
    if m.keywords is not None:
        #cnames = StringVar(value=countrynames)
        kl= Listbox(tl, listvariable=StringVar(value=m.keywords), height=5)
        kl.grid(column=0, row=2, sticky="nsew")
        vs = ttk.Scrollbar(tl, orient=VERTICAL, command=kl.yview)
        kl.configure(yscrollcommand=vs.set)
        vs.grid(column=1, row=2, sticky="nsew")
        hs = ttk.Scrollbar(tl, orient=HORIZONTAL, command=kl.xview)
        kl.configure(xscrollcommand=hs.set)
        hs.grid(column=0, row=3, sticky="nsew")
    rateframe=ttk.Frame(tl, padding="3 3 12 12")
    rateframe.grid(column=0, row=4, sticky="nsew")        
    rating = StringVar()
    if m in ratings:
        rating.set(ratings[m])
    else:
        rating.set("no-opinion")
    ttk.Radiobutton(rateframe, text='Like', variable=rating, value='like').grid(column=0, row=0, sticky="nsew")
    ttk.Radiobutton(rateframe, text='Dislike', variable=rating, value='dislike').grid(column=1, row=0, sticky="nsew")
    ttk.Radiobutton(rateframe, text='No opinion', variable=rating, value='no-opinion').grid(column=2, row=0, sticky="nsew")
    ttk.Button(rateframe, text="Submit", command=rate_movie).grid(column=1, row=1, sticky = "nsew")
        
    

buttonfilterframe = ttk.Frame(filterframe, padding="3 3 12 12")
buttonfilterframe.grid(column=0, row=5, sticky=(N, E))

ttk.Button(buttonfilterframe, text="Filter", command=filter_data).grid(column=0, row=0, sticky = "W")
ttk.Button(buttonfilterframe, text="Reset", command=reset_filters).grid(column=1, row=0, sticky = "E")

# some of the following code is borrowed from https://www.daniweb.com/programming/software-development/threads/350266/creating-table-in-python

data_header = ['Title', 'Year', 'Rating Count', "Mean Rating", 'US Certificate']
#data = [[m.title, m.year, m.num_ratings, m.mean_rating] for m in m_]

data_tree = ttk.Treeview(dataframe, columns=data_header, show="headings", selectmode='browse')

vsb = ttk.Scrollbar(dataframe, orient="vertical", command=data_tree.yview)
hsb = ttk.Scrollbar(dataframe, orient="horizontal", command=data_tree.xview)

data_tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
data_tree.grid(column=0, row=0, sticky='nsew')

data_tree.bind("<Double-1>", double_click)

vsb.grid(column=1, row=0, sticky='ns')
hsb.grid(column=0, row=1, sticky='ew')

def build_a_model_of_the_data(*args):
    print("Not implemented yet")

ttk.Button(dataframe, text="Build a Model of the Data", command=build_a_model_of_the_data).grid(column=0, row=2, sticky = "nsew")


for col in data_header:
    data_tree.heading(col, text=col.title(), command=lambda c=col: sortby(data_tree, c, 0))
    # adjust the column's width to the header string
    data_tree.column(col, width=tkFont.Font().measure(col.title()))
    
for i in range(len(m_)):
    item = [m_[i].title, m_[i].year, m_[i].num_ratings, m_[i].mean_rating, m_[i].certificate]
    data_tree.insert('', 'end', iid = str(i), values=item)
    # adjust column's width if necessary to fit each value
    #for ix, val in enumerate(item):
     #   col_w = tkFont.Font().measure(val)
     #   if data_tree.column(data_header[ix],width=None)<col_w:
     #       data_tree.column(data_header[ix], width=col_w)

root.mainloop()

Not implemented yet
Not implemented yet


In [12]:
ratings = {}

look_for_genres = set()
look_for_genres.update(['Documentary'])

look_for_keywords = set()
look_for_keywords.update(['nature'])

certificates = set()
certificates.update(["G", "PG", "TV-G", "TV-PG"])


for m in m_:
    if m.certificate in certificates:
        if m.genres is not None:
            mgs = set()
            mgs.update(m.genres)
            if len(look_for_genres.intersection(mgs)) == len(look_for_genres):
                #ratings[m] = 'like'
                if m.keywords is not None:
                    mks = set()
                    mks.update(m.keywords)
                    if len(look_for_keywords.intersection(mks)) == len(look_for_keywords):
                        ratings[m] = 'like'

In [13]:
len(ratings)

9

In [27]:
#pickle.dump(ratings, open( "kidsratings.p", "wb" ))

In [14]:
r = []
for m in movies:
    r.append(m.mean_rating)

In [15]:
r_mean = np.mean(r)
r_var = np.var(r)

In [16]:
Dk = []
Dg = []
y = []
for m in m_:
    ms = ''
    if m.keywords is not None:
        ms = " ".join(m.keywords)
    Dk.append(ms)
    mg = ''
    if m.genres is not None:
        mg = " ".join(m.genres)
    Dg.append(mg)
    if m in ratings:
        y.append(1)
    else:
        y.append(0)

In [17]:
len(y)

691

In [18]:
y = np.array(y)

In [19]:
vocabulary = []
vect = CountVectorizer(token_pattern = r'\b\S+\b', lowercase=False)
Xg = vect.fit_transform(Dg)
vocabulary = vect.get_feature_names()
vect = TfidfVectorizer(token_pattern = r'\b\S+\b')
Xk = vect.fit_transform(Dk)
vocabulary = vocabulary + vect.get_feature_names()
X = hstack((Xg, Xk)).tocsr()
X.shape

(691, 9864)

In [20]:
np.array(m_)[np.where(y==1)]

array(["Human Planet"	(2011)	(TV-PG), "Planet Earth II"	(2016)	(TV-G),
       "Planet Earth"	(2006)	(TV-G), "The Blue Planet"	(2001)	(TV-G),
       "The Life of Birds"	(1998)	(TV-PG),
       "Through the Wormhole"	(2010)	(TV-PG),
       "Wonders of Life"	(2013)	(TV-PG),
       "Wonders of the Solar System"	(2010)	(TV-PG),
       "Wonders of the Universe"	(2011)	(TV-PG)], dtype=object)

In [21]:
clf = svm.OneClassSVM(kernel='linear')

In [22]:
clf.fit(X[np.where(y==1)])

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto',
      kernel='linear', max_iter=-1, nu=0.5, random_state=None,
      shrinking=True, tol=0.001, verbose=False)

In [23]:
clf.coef_

<1x9864 sparse matrix of type '<class 'numpy.float64'>'
	with 181 stored elements in Compressed Sparse Row format>

In [24]:
feat_indices = np.argsort(np.abs(clf.coef_.toarray()[0]))[::-1]

In [25]:
for i in feat_indices[:20]:
    print(clf.coef_.toarray()[0][i])

4.5
1.01368736251
0.468534963364
0.408853304338
0.350112905018
0.337195097646
0.323097262956
0.323097262956
0.322950822292
0.317225468001
0.317225468001
0.303056790284
0.303056790284
0.292422244614
0.280333539643
0.279855654104
0.269992162818
0.263546950269
0.263117530996
0.240112752475


In [26]:
for i in feat_indices[:30]:
    print(vocabulary[i])

Documentary
nature
tv-mini-series
nature-documentary
island
bird-watching
urban
primitive
survival
bird-in-title
naturalist
bird
wildlife
science
physicist
peril
british-accent
life
animal-in-title
earth
scientist
planet-earth
biology
professor
bbc
educational
astrophysics
desert
animal
cosmos


In [27]:
positive = np.where(clf.predict(X) != -1)[0]

In [28]:
len(positive)

6

In [29]:
df = (clf.decision_function(X)).flatten()

In [30]:
movie_indices = np.argsort(df)[::-1]

In [31]:
for i in movie_indices[:20]:
    print(m_[i])
    #print(df[i])
    #print(m_[i].keywords)
    #print()

Home	(2009)	(Not Rated)
"The Trials of Life"	(1990)	(Unknown)
"The Blue Planet"	(2001)	(TV-G)
"Planet Earth"	(2006)	(TV-G)
"Planet Earth II"	(2016)	(TV-G)
"Through the Wormhole"	(2010)	(TV-PG)
"Human Planet"	(2011)	(TV-PG)
"Wonders of the Universe"	(2011)	(TV-PG)
"Wonders of Life"	(2013)	(TV-PG)
"Wonders of the Solar System"	(2010)	(TV-PG)
"The Life of Birds"	(1998)	(TV-PG)
"Life on Earth"	(1979)	(Unknown)
"Nature's Great Events"	(2009)	(Unknown)
"Africa"	(2013)	(Unknown)
"The Life of Mammals"	(2002)	(Unknown)
"The National Parks: America's Best Idea"	(2009)	(Unknown)
"Cosmos: A Spacetime Odyssey"	(2014)	(TV-PG)
"Sonic Highways"	(2014)	(Unknown)
"South Pacific"	(2009)	(Unknown)
"Inside Look: The People v. O.J. Simpson, American Crime Story"	(2016)	(Unknown)


In [32]:
df[movie_indices[:20]]

array([  1.27804961e+01,   2.32521573e-03,   5.86136938e-04,
         2.05671834e-04,   1.31987878e-04,   6.34651131e-05,
        -7.02626119e-05,  -9.89617511e-05,  -9.89933241e-05,
        -3.32525384e-04,  -3.86036424e-04,  -4.03890062e-02,
        -5.67618376e-02,  -1.55858125e-01,  -3.64263996e-01,
        -4.44985239e-01,  -5.09812925e-01,  -5.42827183e-01,
        -5.42827183e-01,  -5.42827183e-01])